# Notebook criado para treino de CNN para predição de classes Raio-x ou Não Raio-x

In [0]:
from google.colab import drive
drive.mount('/content/drive')

#### Tutorial utilizado: https://towardsdatascience.com/train-image-recognition-ai-with-5-lines-of-code-8ed0bdd8d9ba

# Instalação do ambiente

In [0]:
!pip3 uninstall tensorflow

In [0]:
!pip3 install tensorflow==1.5

In [0]:
!pip3 install tensorflow-gpu==1.15

In [0]:
!pip3 install keras

In [0]:
!pip3 install imageai --upgrade

# Treino do zero

In [0]:
from imageai.Prediction.Custom import ModelTraining

model_trainer = ModelTraining()
model_trainer.setModelTypeAsInceptionV3()
model_trainer.setDataDirectory("/content/drive/My Drive/Thorax-XRay-Dataset-v2")
model_trainer.trainModel(num_objects=2, num_experiments=100, enhance_data=True, batch_size=4, show_network_summary=True, save_full_model=True)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______

# Treino continuando da 5ª época.

In [0]:
from imageai.Prediction.Custom import ModelTraining

model_trainer = ModelTraining()
model_trainer.setModelTypeAsResNet()
model_trainer.setDataDirectory("/content/drive/My Drive/Thorax-XRay-Dataset")
model_trainer.trainModel(num_objects=2, num_experiments=5, enhance_data=True, batch_size=4, show_network_summary=True, continue_from_model="/content/drive/My Drive/Thorax-XRay-Dataset/models/model_ex-003_acc-0.987342.h5")

# Compactação da Model para download a posteriori

In [0]:
!zip -r Thorax-XRay-Dataset.zip Thorax-XRay-Dataset

# Predição

In [0]:
from imageai.Prediction.Custom import CustomImagePrediction
import os

execution_path = os.getcwd()

prediction = CustomImagePrediction()
prediction.setModelTypeAsResNet()
prediction.setModelPath(os.path.join(execution_path, "drive/My Drive/Thorax-XRay-Dataset/models/model_ex-003_acc-0.992405.h5"))
prediction.setJsonPath(os.path.join(execution_path, "drive/My Drive/Thorax-XRay-Dataset/json/model_class.json"))
prediction.loadModel(num_objects=2)

predictions, probabilities = prediction.predictImage(os.path.join(execution_path, "raiox-maos.jpeg"), result_count=2)

for eachPrediction, eachProbability in zip(predictions, probabilities):
    print(eachPrediction , " : " , eachProbability)

Not-XRay-Thorax  :  64.29535150527954
XRay-Thorax  :  35.70465445518494


# Visualização de especificações de Hardware

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 973369867650267352, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3744122538999160396
 physical_device_desc: "device: XLA_CPU device"]

In [0]:
!cat /proc/cpuinfo
!cat /proc/meminfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.174
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa itlb_multi

# Instalação do driver CUDA para treinar na GPU

In [0]:
!apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/7fa2af80.pub
!wget http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/cuda-repo-ubuntu1604_9.1.85-1_amd64.deb
!apt install ./cuda-repo-ubuntu1604_9.1.85-1_amd64.deb
!wget http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64/nvidia-machine-learning-repo-ubuntu1604_1.0.0-1_amd64.deb
!apt install ./nvidia-machine-learning-repo-ubuntu1604_1.0.0-1_amd64.deb
!apt update

# Install CUDA and tools. Include optional NCCL 2.x
!apt install cuda9.0 cuda-cublas-9-0 cuda-cufft-9-0 cuda-curand-9-0 \
    cuda-cusolver-9-0 cuda-cusparse-9-0 libcudnn7=7.2.1.38-1+cuda9.0 \
    libnccl2=2.2.13-1+cuda9.0 cuda-command-line-tools-9-0

# Optional: Install the TensorRT runtime (must be after CUDA install)
!apt update
!apt install libnvinfer4=4.1.2-1+cuda9.0